In [ ]:
import torch
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install fastapi uvicorn diffusers transformers peft accelerate torch safetensors

In [ ]:
!pip install mistralai

In [ ]:
!pip install pyngrok

In [ ]:
from fastapi import FastAPI, HTTPException, UploadFile, Form ,status , File
from fastapi.responses import FileResponse
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from diffusers import StableDiffusionControlNetPipeline, UNet2DConditionModel, AutoencoderKL, DDIMScheduler , ControlNetModel ,StableDiffusionPipeline , StableDiffusionInpaintPipeline , StableDiffusionControlNetInpaintPipeline
from diffusers import StableDiffusionControlNetImg2ImgPipeline , StableDiffusionImg2ImgPipeline
from transformers import CLIPTextModel, CLIPTokenizer
import torch
import os
import io
import json
import base64
from fastapi.responses import JSONResponse
from diffusers import DiffusionPipeline, EulerDiscreteScheduler
import torch
from diffusers import LCMScheduler, AutoPipelineForText2Image
from diffusers import DiffusionPipeline, AutoencoderKL, UNet2DConditionModel
from transformers import CLIPTextModel, CLIPTextConfig, CLIPTokenizer , CLIPTextModelWithProjection
import torch
import traceback
import asyncio
from concurrent.futures import ThreadPoolExecutor
from fastapi.responses import FileResponse
from io import BytesIO
from fastapi.responses import StreamingResponse
executor = ThreadPoolExecutor()
from fastapi import HTTPException, UploadFile, File
from typing import Dict, Optional
from PIL import Image
from functools import partial
import torch
from torchvision.transforms import ToTensor
from diffusers import (
    StableDiffusionPipeline,
    ControlNetModel, StableDiffusionControlNetPipeline,
    StableDiffusionInpaintPipeline
)
from diffusers import StableDiffusionXLControlNetPipeline

from diffusers import StableDiffusionXLPipeline
# from peft import PeftMode
app = FastAPI()

# Allow cross-origin requests for frontend communication
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # For dev; restrict in production
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


SD15_base_path="runwayml/stable-diffusion-v1-5"
SD15_vae = AutoencoderKL.from_pretrained(SD15_base_path, subfolder="vae", torch_dtype=torch.float16).to("cpu")
SD15_text_encoder = CLIPTextModel.from_pretrained(SD15_base_path, subfolder="text_encoder", torch_dtype=torch.float16).to("cpu")
SD15_tokenizer = CLIPTokenizer.from_pretrained(SD15_base_path, subfolder="tokenizer")
SD15_unet = UNet2DConditionModel.from_pretrained(SD15_base_path, subfolder="unet", torch_dtype=torch.float16).to("cpu")
SD15_scheduler = DDIMScheduler.from_pretrained(SD15_base_path, subfolder="scheduler")

SD15_distilled_path="RajputVansh/SD_15"
SD15D_vae = AutoencoderKL.from_pretrained(SD15_distilled_path, subfolder="vae", torch_dtype=torch.float16).to("cpu")
SD15D_text_encoder = CLIPTextModel.from_pretrained(SD15_distilled_path, subfolder="text_encoder", torch_dtype=torch.float16).to("cpu")
SD15D_tokenizer = CLIPTokenizer.from_pretrained(SD15_distilled_path, subfolder="tokenizer")
SD15D_unet = UNet2DConditionModel.from_pretrained(SD15_distilled_path, subfolder="unet", torch_dtype=torch.float16).to("cpu")
SD15D_scheduler = DDIMScheduler.from_pretrained(SD15_distilled_path, subfolder="scheduler")

# Load individual components
SDXL_base_path="stabilityai/stable-diffusion-xl-base-1.0"
SDXL_vae = AutoencoderKL.from_pretrained(
    SDXL_base_path,
    subfolder="vae",
    variant="fp16",
    torch_dtype=torch.float16
).to("cpu")

SDXL_unet = UNet2DConditionModel.from_pretrained(
    SDXL_base_path,
    subfolder="unet",
    variant="fp16",
    torch_dtype=torch.float16
).to("cpu")


SDXL_text_encoder = CLIPTextModel.from_pretrained(
    SDXL_base_path,
    subfolder="text_encoder",
    torch_dtype=torch.float16
).to("cpu")
# SDXL uses two text encoders
SDXL_text_encoder_two = CLIPTextModelWithProjection.from_pretrained(
    SDXL_base_path,
    subfolder="text_encoder_2",
    torch_dtype=torch.float16
).to("cpu")


SDXL_tokenizer_one = CLIPTokenizer.from_pretrained(
    SDXL_base_path,
    subfolder="tokenizer"
)

SDXL_tokenizer_two = CLIPTokenizer.from_pretrained(
    SDXL_base_path,
    subfolder="tokenizer_2"
)

SDXL_scheduler = EulerDiscreteScheduler.from_pretrained(
    SDXL_base_path,
    subfolder="scheduler"
)



SDXL_distilled_path="RajputVansh/SDXL-1B-NEW"
SDXLD_vae = AutoencoderKL.from_pretrained(
    SDXL_distilled_path,
    subfolder="vae",
    # variant="fp16",
    torch_dtype=torch.float16
).to("cpu")

SDXLD_unet = UNet2DConditionModel.from_pretrained(
    SDXL_distilled_path,
    subfolder="unet",
    # variant="fp16",
    torch_dtype=torch.float16
).to("cpu")


SDXLD_text_encoder = CLIPTextModel.from_pretrained(
    SDXL_distilled_path,
    subfolder="text_encoder",
    torch_dtype=torch.float16
).to("cpu")
# SDXL uses two text encoders
SDXLD_text_encoder_two = CLIPTextModelWithProjection.from_pretrained(
    SDXL_distilled_path,
    subfolder="text_encoder_2",
    torch_dtype=torch.float16
).to("cpu")


SDXLD_tokenizer_one = CLIPTokenizer.from_pretrained(
    SDXL_distilled_path,
    subfolder="tokenizer"
)

SDXLD_tokenizer_two = CLIPTokenizer.from_pretrained(
    SDXL_distilled_path,
    subfolder="tokenizer_2"
)

SDXLD_scheduler = EulerDiscreteScheduler.from_pretrained(
    SDXL_distilled_path,
    subfolder="scheduler"
)
"""generation mode ham frontend se confirm karenge"""
# generation_mode="inpainting/outpaiting"
# if generation_mode=="txt2img":
pipe = StableDiffusionPipeline(
    vae=SD15_vae,
    text_encoder=SD15_text_encoder,
    tokenizer=SD15_tokenizer,
    unet=SD15_unet,
    scheduler=SD15_scheduler,
    safety_checker=None,
    feature_extractor=None,
    requires_safety_checker=False,
).to("cpu")

print(pipe)
from typing import Optional, Dict
active_controlnets: Dict[str, ControlNetModel] = {}  # Tracks loaded ControlNets

# Global store for frontend inputs
stored_inputs: Dict[str, Dict[str, Optional[str]]] = {}

@app.post("/load-controlnet/")
async def load_controlnet(
    controlnet_path: str = Form(..., description="Path to ControlNet/ControlLoRA model"),
    adapter_name: Optional[str] = Form(None, description="Unique name for model management"),
    lora_weights_path: Optional[str] = Form(None, description="Required for ControlLoRA"),
    torch_dtype: Optional[str] = Form("float16", description="float16 or float32")
):
    # Generate a unique key to identify this input
    model_key = "controlnet"

    # Store input values in global variable
    stored_inputs[model_key] = {
        "controlnet_path": controlnet_path,
        "adapter_name": adapter_name,
        "lora_weights_path": lora_weights_path,
        "torch_dtype": torch_dtype
    }

    return {"status": "stored", "adapter_name": model_key}

app.state.pipeline = None
@app.post("/generate-image/")
async def generate(
    prompt: str = Form(...),
    height: int = Form(...),
    width: int = Form(...),
    num_inference_steps: int = Form(...),
    guidance_scale: float = Form(...),
    model: str = Form(...),  # 👈 Add this line
    control_image: Optional[UploadFile] = File(None),
    init_image: Optional[UploadFile] = File(None),
    strength: Optional[float] = Form(0.8)
):
    print("Prompt:", prompt)
    print("Height:", height)
    print("Width:", width)
    print("Num Inference Steps:", num_inference_steps)
    print("Guidance Scale:", guidance_scale)
    print("Control Image:", control_image.filename if control_image else None)
    print("Init Image:", init_image.filename if init_image else None)
    print("checkpoint 0")
    print("model" , model)
    try:
        print("checkpoint 1")
        # print(f"⚙️ Generating with ControlNet: {pipe.controlnet is not None}")
        # Suppose you want to retrieve a specific model's inputs:
        model_key = "controlnet"  # Use the key you stored it with
        if model_key in stored_inputs:
            controlnet_path = stored_inputs[model_key]["controlnet_path"]
            adapter_name = stored_inputs[model_key]["adapter_name"]
            torch_dtype="float16"
            print(f"ControlNet path: {controlnet_path}")
            print(f"Adapter name: {adapter_name}")
            adapter_name = f"controlnet_{adapter_name or os.path.basename(controlnet_path)}"
            # Validate inputs
            dtype = torch.float16 if torch_dtype == "float16" else torch.float32
            # Load ControlNet/ControlLoRA
            controlnet = ControlNetModel.from_pretrained(
                controlnet_path,
                torch_dtype=dtype
            ).to("cuda")
            model_key = adapter_name or controlnet_path
            active_controlnets[model_key] = controlnet
        else:
            controlnet=None
            dtype=None
            print(f"No stored input found for key: {model_key}")
        # base_model_path="runwayml/stable-diffusion-v1-5"
        if controlnet is not None and control_image is not None:
                print("Entered block with CN and CI")
                if model == "SD1.5_Base":
                  print("SD1.5 Base")
                  SD15_vae.to("cuda")
                  SD15_text_encoder.to("cuda")
                  SD15_unet.to("cuda")
                  pipe = StableDiffusionControlNetPipeline(
              vae=SD15_vae, text_encoder=SD15_text_encoder, tokenizer=SD15_tokenizer,
              unet=SD15_unet, controlnet=controlnet, scheduler=SD15_scheduler,
              safety_checker=None, feature_extractor=None
          ).to("cuda")
                elif model == "SDXL_Base":
                  print("SDXLBase")
                  SDXL_vae.to("cuda")
                  SDXL_unet.to("cuda")
                  SDXL_text_encoder.to("cuda")
                  SDXL_text_encoder_two.to("cuda")
                  pipe = StableDiffusionXLControlNetPipeline(
                        vae=SDXL_vae,
                        text_encoder=SDXL_text_encoder,
                        text_encoder_2=SDXL_text_encoder_two,
                        tokenizer=SDXL_tokenizer_one,
                        tokenizer_2=SDXL_tokenizer_two,
                        controlnet=controlnet,
                        unet=SDXL_unet,
                        scheduler=SDXL_scheduler
                    ).to("cuda")
                elif model == "SD1.5_Distilled":
                  print("SD1.5_Distilled")
                  SD15D_vae.to("cuda")
                  SD15D_text_encoder.to("cuda")
                  SD15D_unet.to("cuda")
                  pipe = StableDiffusionControlNetPipeline(
                        vae=SD15D_vae, text_encoder=SD15D_text_encoder, tokenizer=SD15D_tokenizer,
                        unet=SD15D_unet, controlnet=controlnet, scheduler=SD15D_scheduler,
                        safety_checker=None, feature_extractor=None
                    ).to("cuda")
                elif model == "SDXL_Distilled":
                  print("SDXL_Distilled")
                  SDXLD_vae.to("cuda")
                  SDXLD_unet.to("cuda")
                  SDXLD_text_encoder.to("cuda")
                  SDXLD_text_encoder_two.to("cuda")
                  pipe = StableDiffusionXLControlNetPipeline(
                        vae=SDXLD_vae,
                        text_encoder=SDXLD_text_encoder,
                        text_encoder_2=SDXLD_text_encoder_two,
                        tokenizer=SDXLD_tokenizer_one,
                        tokenizer_2=SDXLD_tokenizer_two,
                        controlnet=controlnet,
                        unet=SDXLD_unet,
                        scheduler=SDXLD_scheduler
                    ).to("cuda")
                # pipe = compile(pipe, config)
        # Case 2: No ControlNet, just base model
        elif controlnet is None:
            print("Entered block without CN")
            if model == "SD1.5_Base":
              print("SD1.5 Base")
              SD15_vae.to("cuda")
              SD15_text_encoder.to("cuda")
              SD15_unet.to("cuda")
              pipe = StableDiffusionPipeline(
                    vae=SD15_vae,
                    text_encoder=SD15_text_encoder,
                    tokenizer=SD15_tokenizer,
                    unet=SD15_unet,
                    scheduler=SD15_scheduler,
                    safety_checker=None,
                    feature_extractor=None,
                    requires_safety_checker=False,
                ).to("cuda")
            elif model == "SDXL_Base":
                  print("SDXLBase")
                  SDXL_vae.to("cuda")
                  SDXL_unet.to("cuda")
                  SDXL_text_encoder.to("cuda")
                  SDXL_text_encoder_two.to("cuda")
                  pipe = StableDiffusionXLPipeline(
                        vae=SDXL_vae,
                        text_encoder=SDXL_text_encoder,
                        text_encoder_2=SDXL_text_encoder_two,
                        tokenizer=SDXL_tokenizer_one,
                        tokenizer_2=SDXL_tokenizer_two,
                        unet=SDXL_unet,
                        scheduler=SDXL_scheduler
                    ).to("cuda")
            elif model == "SD1.5_Distilled":
              print("SD1.5_Distilled")
              SD15D_vae.to("cuda")
              SD15D_text_encoder.to("cuda")
              SD15D_unet.to("cuda")
              pipe = StableDiffusionPipeline(
                    vae=SD15D_vae, text_encoder=SD15D_text_encoder, tokenizer=SD15D_tokenizer,
                    unet=SD15D_unet, scheduler=SD15D_scheduler,
                    safety_checker=None, feature_extractor=None
                ).to("cuda")
            elif model == "SDXL_Distilled":
              print("SDXL_Distilled")
              SDXLD_vae.to("cuda")
              SDXLD_unet.to("cuda")
              SDXLD_text_encoder.to("cuda")
              SDXLD_text_encoder_two.to("cuda")
              pipe = StableDiffusionXLPipeline(
                    vae=SDXLD_vae,
                    text_encoder=SDXLD_text_encoder,
                    text_encoder_2=SDXLD_text_encoder_two,
                    tokenizer=SDXLD_tokenizer_one,
                    tokenizer_2=SDXLD_tokenizer_two,
                    unet=SDXLD_unet,
                    scheduler=SDXLD_scheduler
                ).to("cuda")
            # pipe = compile(pipe, config)
        # Case 3: ControlNet but no Control Image – invalid
        elif controlnet is not None and control_image is None:
            raise ValueError("Control image must be provided when using ControlNet.")
            print("Will Proceed to Generate image with the base model")
            pipe=pipe
        print(pipe)
        app.state.pipeline = pipe
        if controlnet is not None and control_image is not None:
          current_time=time.time()
          def generate_image_sync(prompt, height, width, num_inference_steps, guidance_scale,control_image_pil=None):
              try:
                generator = torch.manual_seed(42)
                gen_start_time = time.time()
                return pipe(
                    prompt=prompt,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    image=control_image_pil,
                    generator=generator,
                ).images[0]
                gen_end_time = time.time()
                print(f"🕒 Image generation took: {gen_end_time - gen_start_time:.2f} seconds")
              except Exception as e:
                    print("❌ Exception inside generate_image_sync:", e)
                    import traceback
                    traceback.print_exc()
                    raise


          # Process control image if provided
          print("Checkpoint fnxn pass")
          control_image_pil = None
          if control_image:
              image_bytes = await control_image.read()
              control_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
              # control_image_pil = ToTensor()(control_image_pil).unsqueeze(0)  # Add batch dimension
          #     print("chckpnt23")
          #     print("Control image tensor device: %s", control_image_pil.device)
              # control_image_pil = control_image_pil.to("cuda")  # Move to GPU
          #     # control_image_pil = Image.open(BytesIO(await control_image.read()))
          print("checkpoint 2")
          print("🧪 Type of control_image_pil:", type(control_image_pil))
          # print("🧾 Is controlnet loaded?", pipe.controlnet is not None)
          print("🖼️ Final control image type:", type(control_image_pil))
          # print("Control image tensor device: %s", control_image_pil.device)
          # # Run in executor
          fn = partial(
                        generate_image_sync,
                        prompt=prompt,
                        height=height,
                        width=width,
                        num_inference_steps=num_inference_steps,
                        guidance_scale=guidance_scale,
                        control_image_pil=control_image_pil  # Explicitly name the parameter
                    )
          # SD15_vae.to("cpu")
          # SD15_text_encoder.to("cpu")
          # SD15_unet.to("cpu")
          # del pipe
          # torch.cuda.empty_cache()
          end_time=time.time()
          time_taken=end_time-current_time
        elif controlnet is None:
          def generate_image_sync(prompt, height, width, num_inference_steps, guidance_scale):
            try:
                generator = torch.manual_seed(42)
                return pipe(
                    prompt=prompt,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    generator=generator
                ).images[0]
            except Exception as e:
                print("❌ Error in text2img:", e)
                raise
            # Process control image if provided
          print("Checkpoint fnxn pass")
          control_image_pil = None
          if control_image:
              image_bytes = await control_image.read()
              control_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
          print("checkpoint 2")
          print("🧪 Type of control_image_pil:", type(control_image_pil))
          # print("🧾 Is controlnet loaded?", pipe.controlnet is not None)
          print("🖼️ Final control image type:", type(control_image_pil))
          fn = partial(
                    generate_image_sync,
                    prompt=prompt,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale  # Explicitly name the parameter
                )
          # SD15_vae.to("cpu")
          # SD15_text_encoder.to("cpu")
          # SD15_unet.to("cpu")
          # del pipe
          # torch.cuda.empty_cache()
        elif controlnet is not None and control_image is None:
          def generate_image_sync(prompt, height, width, num_inference_steps, guidance_scale):
            try:
                generator = torch.manual_seed(42)
                return pipe(
                    prompt=prompt,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    generator=generator
                ).images[0]
            except Exception as e:
                print("❌ Error in text2img:", e)
                raise
            control_image_pil = None
          if control_image:
              image_bytes = await control_image.read()
              control_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
          print("checkpoint 2")
          print("🧪 Type of control_image_pil:", type(control_image_pil))
          # print("🧾 Is controlnet loaded?", pipe.controlnet is not None)
          print("🖼️ Final control image type:", type(control_image_pil))
          fn = partial(
                    generate_image_sync,
                    prompt=prompt,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale  # Explicitly name the parameter
                )
          # del pipe
          # torch.cuda.empty_cache()
        image = await asyncio.get_event_loop().run_in_executor(executor, fn)
        SD15_vae.to("cpu")
        SD15_text_encoder.to("cpu")
        SD15_unet.to("cpu")
        SDXL_vae.to("cpu")
        SDXL_unet.to("cpu")
        SDXL_text_encoder.to("cpu")
        SDXL_text_encoder_two.to("cpu")
        SDXLD_vae.to("cpu")
        SDXLD_unet.to("cpu")
        SDXLD_text_encoder.to("cpu")
        SDXLD_text_encoder_two.to("cpu")
        SD15D_vae.to("cpu")
        SD15D_text_encoder.to("cpu")
        SD15D_unet.to("cpu")
        del pipe
        torch.cuda.empty_cache()
        print("checkpoint 3")
        # Stream response
        img_bytes = BytesIO()
        image.save(img_bytes, format="PNG")
        img_bytes.seek(0)
        return StreamingResponse(
            img_bytes,
            media_type="image/png",
            headers={"Content-Disposition": 'inline; filename="output.png"'}
        )
    except Exception as e:
        print(f"❌ Generation failed: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))
@app.post("/generate-image-inpainting/")
async def generate( prompt: str = Form(...),
    height: int = Form(...),
    width: int = Form(...),
    num_inference_steps: int = Form(...),
    guidance_scale: float = Form(...),
    control_image: Optional[UploadFile] = File(None),
    init_image: Optional[UploadFile] = File(None),
    strength: Optional[float] = Form(0.8)):
    print("inside the new endpoint")
    print("Prompt:", prompt)
    print("Height:", height)
    print("Width:", width)
    print("Num Inference Steps:", num_inference_steps)
    print("Guidance Scale:", guidance_scale)
    print("Control Image:", control_image.filename if control_image else None)
    print("Init Image:", init_image.filename if control_image else None)
    print("checkpoint 0")
    try:
        print("checkpoint 1")
        print("Inpaiting Mode")
        controlnet=None
        """for now we are hard coding this , till end eval we will also add this"""
        if controlnet is not None and control_image is not None:
                print("Entered block with CN and CI")
                SD15_vae.to("cuda")
                SD15_text_encoder.to("cuda")
                SD15_unet.to("cuda")
                pipe = StableDiffusionControlNetInpaintPipeline(
            vae=SD15_vae, text_encoder=SD15_text_encoder, tokenizer=SD15_tokenizer,
            unet=SD15_unet, controlnet=controlnet, scheduler=SD15_scheduler,
            safety_checker=None, feature_extractor=None
        ).to("cuda")
                # pipe = compile(pipe, config)
        # Case 2: No ControlNet, just base model
        elif controlnet is None:
            print("Entered block without CN")
            SD15_vae.to("cuda")
            SD15_text_encoder.to("cuda")
            SD15_unet.to("cuda")
            pipe = StableDiffusionInpaintPipeline(
                  vae=SD15_vae,
                  text_encoder=SD15_text_encoder,
                  tokenizer=SD15_tokenizer,
                  unet=SD15_unet,
                  scheduler=SD15_scheduler,
                  safety_checker=None,
                  feature_extractor=None,
                  requires_safety_checker=False,
              ).to("cuda")
            # pipe = compile(pipe, config)
        # Case 3: ControlNet but no Control Image – invalid
        elif controlnet is not None and control_image is None:
            raise ValueError("Control image must be provided when using ControlNet.")
            print("Will Proceed to Generate image with the base model")
            print("Entered block without CN")
            SD15_vae.to("cuda")
            SD15_text_encoder.to("cuda")
            SD15_unet.to("cuda")
            pipe = StableDiffusionInpaintPipeline(
                  vae=SD15_vae,
                  text_encoder=SD15_text_encoder,
                  tokenizer=SD15_tokenizer,
                  unet=SD15_unet,
                  scheduler=SD15_scheduler,
                  safety_checker=None,
                  feature_extractor=None,
                  requires_safety_checker=False,
              ).to("cuda")
        print(pipe)
        app.state.pipeline = pipe
        if controlnet is not None and control_image is not None:
          def generate_image_sync(prompt, init_image_pil, strength, num_inference_steps, guidance_scale, control_image_pil ):
            try:
                generator = torch.manual_seed(42)
                return pipe(
                    prompt=prompt,
                    image=init_image_pil,
                    strength=strength,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    control_image=control_image_pil,
                    generator=generator
                ).images[0]
            except Exception as e:
                print("❌ Error in img2img with ControlNet:", e)
                raise
          control_image_pil = None
          if control_image:
              image_bytes = await control_image.read()
              control_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
          init_image_pil = None
          if init_image:
              image_bytes = await init_image.read()
              init_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
          print("checkpoint 2")
          print("🧪 Type of control_image_pil:", type(control_image_pil))
          # print("🧾 Is controlnet loaded?", pipe.controlnet is not None)
          print("🖼️ Final control image type:", type(control_image_pil))
          fn = partial(
                    generate_image_sync,
                    prompt=prompt,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    control_image_pil=control_image_pil,
                    init_image_pil=init_image_pil,
                    strength=strength,# Explicitly name the parameter
                )
          torch.cuda.empty_cache()
        elif controlnet is None:
          def generate_image_sync(prompt,init_image_pil,mask_image_pil, height, width, num_inference_steps, guidance_scale,strength=0.6):
            try:
                generator = torch.manual_seed(42)
                return pipe(
                    prompt=prompt,
                    image=init_image_pil,
                    mask_image=mask_image_pil,
                    strength=strength,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    generator=generator,

                ).images[0]
            except Exception as e:
                print("❌ Error in inpainting:", e)
                raise
          print("Init Image:", init_image.filename if init_image else None)
          init_image_pil = None
          if init_image:
              image_bytes = await init_image.read()
              init_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
          control_image_pil = None
          if control_image:
              print("control image is present")
              image_bytes = await control_image.read()
              mask_image_pil = Image.open(BytesIO(image_bytes)).convert("L")
              mask_image_pil = mask_image_pil.resize(init_image_pil.size, Image.NEAREST)
          print("checkpoint 2")
          print("🧪 Type of init_image_pil:", type(control_image_pil))
          # print("🧾 Is controlnet loaded?", pipe.controlnet is not None)
          print("🖼️ Final init image type:", type(control_image_pil))
          fn = partial(
                    generate_image_sync,
                    prompt=prompt,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    mask_image_pil=mask_image_pil,
                    init_image_pil=init_image_pil,
                    strength=strength # Explicitly name the parameter
                )
          torch.cuda.empty_cache()
        elif controlnet is not None and control_image is None:
          def generate_image_sync(prompt,init_image_pil, height, width, num_inference_steps, guidance_scale,strength=0.6):
            try:
                generator = torch.manual_seed(42)
                return pipe(
                    prompt=prompt,
                    image=init_image_pil,
                    strength=strength,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    generator=generator
                ).images[0]
            except Exception as e:
                print("❌ Error in text2img:", e)
                raise
          if control_image:
              image_bytes = await control_image.read()
              control_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
          init_image_pil = None
          if init_image:
              image_bytes = await init_image.read()
              init_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
          print("checkpoint 2")
          print("🧪 Type of control_image_pil:", type(control_image_pil))
          # print("🧾 Is controlnet loaded?", pipe.controlnet is not None)
          print("🖼️ Final control image type:", type(control_image_pil))
          fn = partial(
                    generate_image_sync,
                    prompt=prompt,
                    height=height,
                    width=width,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=guidance_scale,
                    init_image_pil=init_image_pil,
                    strength=strength, # Explicitly name the parameter
                )

          torch.cuda.empty_cache()
        image = await asyncio.get_event_loop().run_in_executor(executor, fn)
        SD15_vae.to("cpu")
        SD15_text_encoder.to("cpu")
        SD15_unet.to("cpu")
        del pipe
        print("checkpoint 3")
        # Stream response
        img_bytes = BytesIO()
        image.save(img_bytes, format="PNG")
        img_bytes.seek(0)
        return StreamingResponse(
            img_bytes,
            media_type="image/png",
            headers={"Content-Disposition": 'inline; filename="output.png"'}
        )
    except Exception as e:
        print(f"❌ Generation failed: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))
@app.post("/generate-img2img-image/")
async def generate( prompt: str = Form(...),
    base_model: str = Form(...),             # path or name (e.g. "stabilityai/stable-diffusion-2-1")
    lora_adapter: Optional[str] = Form(None), # path to .safetensors or other adapter
    controlnet_path: Optional[str] = Form(None), # path to controlnet weights
    denoising_strength: float = Form(0.8),
    cfg_scale: float = Form(7.5),
    num_inference_steps: int = Form(50),
    seed: Optional[int] = Form(None),
    init_image: UploadFile = File(...),
    control_image: Optional[UploadFile] = File(None)):
    print("Prompt:", prompt)
    print("Num Inference Steps:", num_inference_steps)
    print("Strength:",denoising_strength)
    print("Guidance Scale:", cfg_scale)
    print("Control Image:", control_image.filename if control_image else None)
    print("Init Image:", init_image.filename if control_image else None)
    print("checkpoint 0")
    if controlnet_path:
      controlnet = ControlNetModel.from_pretrained(
                  controlnet_path,
                  torch_dtype=torch.float16
              ).to("cuda")
    else:
      controlnet=None
    try:
        if controlnet is not None and control_image is not None:
            print("Entered block with CN and CI")
            SD15_vae.to("cuda")
            SD15_text_encoder.to("cuda")
            SD15_unet.to("cuda")
            pipe = StableDiffusionControlNetImg2ImgPipeline(
            vae=SD15_vae, text_encoder=SD15_text_encoder, tokenizer=SD15_tokenizer,
            unet=SD15_unet, controlnet=controlnet, scheduler=SD15_scheduler,
            safety_checker=None, feature_extractor=None
        ).to("cuda")
        elif controlnet is None:
            print("Entered block without CN")
            SD15_vae.to("cuda")
            SD15_text_encoder.to("cuda")
            SD15_unet.to("cuda")
            pipe = StableDiffusionImg2ImgPipeline(
                  vae=SD15_vae,
                  text_encoder=SD15_text_encoder,
                  tokenizer=SD15_tokenizer,
                  unet=SD15_unet,
                  scheduler=SD15_scheduler,
                  safety_checker=None,
                  feature_extractor=None,
                  requires_safety_checker=False,
              ).to("cuda")
            # pipe = compile(pipe, config)
        # Case 3: ControlNet but no Control Image – invalid
        app.state.pipeline = pipe
        if controlnet is not None and control_image is not None:
          def generate_image_sync(prompt, init_image_pil, denoising_strength, num_inference_steps, cfg_scale,control_image_pil ):
            try:
                generator = torch.manual_seed(42)
                return pipe(
                    prompt=prompt,
                    image=init_image_pil,
                    strength=denoising_strength,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=cfg_scale,
                    control_image=control_image_pil,
                    generator=generator
                ).images[0]
            except Exception as e:
                print("❌ Error in img2img with ControlNet:", e)
                raise
          control_image_pil = None
          if control_image:
              image_bytes = await control_image.read()
              control_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
              # control_image_pil = control_image_pil.resize((width, height), Image.LANCZOS)
          init_image_pil = None
          if init_image:
              image_bytes = await init_image.read()
              init_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
              # init_image_pil = init_image_pil.resize((width, height), Image.LANCZOS)
          print("checkpoint 2")
          print("🧪 Type of control_image_pil:", type(control_image_pil))
          # print("🧾 Is controlnet loaded?", pipe.controlnet is not None)
          print("🖼️ Final control image type:", type(control_image_pil))
          fn = partial(
                    generate_image_sync,
                    prompt=prompt,
                    # height=height,
                    # width=width,
                    num_inference_steps=num_inference_steps,
                    cfg_scale=cfg_scale,
                    control_image_pil=control_image_pil,
                    init_image_pil=init_image_pil,
                    denoising_strength=denoising_strength,# Explicitly name the parameter
                )

        elif controlnet is None:
          def generate_image_sync(prompt,init_image_pil,control_image_pil,num_inference_steps, cfg_scale,denoising_strength):
            try:
                generator = torch.manual_seed(42)
                return pipe(
                    prompt=prompt,
                    image=init_image_pil,
                    mask_image=control_image_pil,
                    strength=denoising_strength,
                    num_inference_steps=num_inference_steps,
                    guidance_scale=cfg_scale,
                    generator=generator,

                ).images[0]
            except Exception as e:
                print("❌ Error in inpainting:", e)
                raise
          print("Init Image:", init_image.filename if init_image else None)
          init_image_pil = None
          if init_image:
              image_bytes = await init_image.read()
              init_image_pil = Image.open(BytesIO(image_bytes)).convert("RGB")
              # init_image_pil = init_image_pil.resize((width, height), Image.LANCZOS)
          control_image_pil = None
          if control_image:
              image_bytes = await control_image.read()
              control_image_pil = Image.open(BytesIO(image_bytes)).convert("L")
              # control_image_pil = control_image_pil.resize((width, height), Image.LANCZOS)
              control_image_pil = control_image_pil.resize(init_image_pil.size, Image.NEAREST)
          print("checkpoint 2")
          print("🧪 Type of init_image_pil:", type(init_image_pil))
          # print("🧾 Is controlnet loaded?", pipe.controlnet is not None)
          print("🖼️ Final init image type:", type(init_image_pil))
          fn = partial(
                    generate_image_sync,
                    prompt=prompt,
                    num_inference_steps=num_inference_steps,
                    cfg_scale=cfg_scale,
                    control_image_pil=control_image_pil,
                    init_image_pil=init_image_pil,
                    denoising_strength=denoising_strength  # Explicitly name the parameter
                )
        image = await asyncio.get_event_loop().run_in_executor(executor, fn)
        SD15_vae.to("cpu")
        SD15_text_encoder.to("cpu")
        SD15_unet.to("cpu")
        del pipe
        torch.cuda.empty_cache()
        print("checkpoint 3")
        # Stream response
        img_bytes = BytesIO()
        image.save(img_bytes, format="PNG")
        img_bytes.seek(0)
        return StreamingResponse(
            img_bytes,
            media_type="image/png",
            headers={"Content-Disposition": 'inline; filename="output.png"'}
        )
    except Exception as e:
        print(f"❌ Generation failed: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))
@app.get("/active-controlnets/")
async def list_controlnets():
    return {"active_controlnets": list(active_controlnets.keys())}
@app.delete("/unload-controlnet/")
async def unload_controlnet(
    model_key: str = Form(..., description="Name or path of model to unload")
):
    try:
        # if model_key in active_controlnets:
        # del active_controlnets[model_key]
        del active_controlnets
        return {"message": f"Unloaded {model_key}"}
        # return {"message": "Model not found"}, 404
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/pipeline-state/")
async def get_state():
    pipeline = app.state.pipeline
    if not pipeline:
        raise HTTPException(404, "Pipeline not initialized")

    response = {
        "pipeline_type": pipeline.__class__.__name__,
        "active_controlnet": False,  # Default value
        "active_adapters": []
    }

    # Safe controlnet check
    if hasattr(pipeline, 'controlnet'):
        response["active_controlnet"] = pipeline.controlnet is not None

    # Safe adapters check
    if hasattr(pipeline, 'get_active_adapters'):
        try:
            response["active_adapters"] = list(pipeline.get_active_adapters())
        except Exception as e:
            response["adapter_error"] = str(e)

    return response

import os
import re
from fastapi import Form, HTTPException
from huggingface_hub import snapshot_download

@app.post("/load-lora/")
async def load_lora(
    lora_path: str = Form(...),
    adapter_name: str = Form("default"),
    lora_scale: float = Form(1.0)
    ):
    try:
        adapter_name=f"lora_{adapter_name}"
        # Check if it's a Hugging Face repo path (contains '/')
        if '/' in lora_path:
            # Download/cache the repository locally
            local_path = snapshot_download(repo_id=lora_path, repo_type="model")
        else:
            # It's already a local path
            local_path = lora_path

        # Find all safetensors files in the directory
        safetensors_files = [f for f in os.listdir(local_path) if f.endswith('.safetensors')]

        if not safetensors_files:
            raise HTTPException(status_code=404, detail="No safetensors files found in the specified path")

        # If only one file, use it directly
        if len(safetensors_files) == 1:
            best_file = safetensors_files[0]
        else:
            # Function to extract step number from filename
            def extract_step_number(fname):
                # Look for patterns like: -000032.safetensors, -1500.safetensors, etc.
                match = re.search(r"-(\d+)\.safetensors$", fname)
                return int(match.group(1)) if match else -1

            # Select the file with the highest step number
            best_file = max(safetensors_files, key=extract_step_number)

            # Delete all other safetensors files, keep only the best one
            for file in safetensors_files:
                if file != best_file:
                    file_path = os.path.join(local_path, file)
                    try:
                        os.remove(file_path)
                        print(f"🗑️ Deleted: {file}")
                    except Exception as delete_error:
                        print(f"⚠️ Warning: Could not delete {file}: {delete_error}")

        # Construct full path to the best file
        best_file_path = os.path.join(local_path, best_file)

        print(f"🚀 Loading LoRA: {best_file_path}")
        if not hasattr(pipe, "load_lora_weights"):
            raise HTTPException(status_code=400, detail="Pipeline doesn't support LoRA")
        # Load the LoRA weights
        pipe.load_lora_weights(local_path,adapter_name=adapter_name)

        return {
            "message": f"Successfully loaded {best_file} from {lora_path}",
            "loaded_file": best_file,
            "local_path": local_path
        }

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Failed to load LoRA: {str(e)}")
import psutil

def get_cpu_ram_stats():
    ram = psutil.virtual_memory()
    cpu = psutil.cpu_percent(interval=1)
    return {
        "cpu_percent": cpu,
        "ram_total": ram.total,
        "ram_used": ram.used,
        "ram_percent": ram.percent
    }
import subprocess
import re

def get_gpu_stats():
    try:
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=memory.total,memory.used,utilization.gpu', '--format=csv,nounits,noheader']
        )
        total, used, utilization = map(int, result.decode().strip().split(', '))
        return {
            "gpu_memory_total": total,
            "gpu_memory_used": used,
            "gpu_utilization": utilization
        }
    except Exception as e:
        return {"error": str(e)}
@app.get("/stats")
def get_stats():
    return {
        "cpu_ram": get_cpu_ram_stats(),
        "gpu": get_gpu_stats()
    }
import os
from mistralai import Mistral

# Set up Mistral API
api_key = os.environ.get("MISTRAL_API_KEY") or "MhesL6wAACd02SEfFnae2cUgmyC3eyxR"
client = Mistral(api_key=api_key)

# Comprehensive dictionary of available LoRAs with their trigger words and descriptions
AVAILABLE_LORAS = {
    "TheLastBen/Papercut_SDXL": {
        "trigger_words": ["papercut style", "papercut art"],
        "description": "Creates papercut art style with layered, cut-paper aesthetic. Perfect for craft-like, dimensional artwork with clean edges and layered appearance."
    },
    "ntc-ai/SDXL-LoRA-slider.extremely-detailed": {
        "trigger_words": [],
        "description": "Enhances image detail and sharpness significantly. Adds intricate textures, fine details, and overall visual complexity to any subject."
    },
    "ntc-ai/SDXL-LoRA-slider.ultra-realistic-illustration": {
        "trigger_words": [],
        "description": "Creates photorealistic illustrations with high fidelity. Ideal for portraits, realistic character art, and lifelike renderings."
    },
    "ostris/crayon_style_lora_sdxl": {
        "trigger_words": ["crayon style", "crayon drawing"],
        "description": "Mimics crayon drawing aesthetics with waxy textures, soft blending, and childlike artistic charm. Great for whimsical, hand-drawn looks."
    },
    "RalFinger/alien-style-lora-sdxl": {
        "trigger_words": ["alienzkin"],
        "description": "Transforms subjects with alien-like characteristics, otherworldly features, and sci-fi aesthetics. Adds extraterrestrial and futuristic elements."
    },
    "ntc-ai/SDXL-LoRA-slider.Studio-Ghibli-style": {
        "trigger_words": ["Studio Ghibli style"],
        "description": "Recreates the distinctive Studio Ghibli animation style with soft colors, whimsical characters, and magical atmosphere reminiscent of Miyazaki films."
    },
    "Norod78/sdxl-chalkboarddrawing-lora": {
        "trigger_words": ["ChalkBoardDrawing"],
        "description": "Creates chalkboard drawing aesthetics with white chalk on dark background, educational/academic feel, and hand-drawn instructional appearance."
    },
    "Norod78/SDXL-JojosoStyle-Lora-v2": {
        "trigger_words": ["JojosoStyle"],
        "description": "Recreates the distinctive JoJo's Bizarre Adventure manga/anime art style with dramatic poses, bold lines, and flamboyant character designs."
    },
    "ntc-ai/SDXL-LoRA-slider.huge-anime-eyes": {
        "trigger_words": ["huge anime eyes"],
        "description": "Enhances anime characters with oversized, expressive eyes typical of manga/anime art. Perfect for kawaii and moe-style characters."
    },
    "Norod78/SDXL-Psychemelt-style-LoRA": {
        "trigger_words": ["psychemelt style", "in an LSD trip psychemelt style"],
        "description": "Creates psychedelic, melting, surreal visuals with warped reality effects. Ideal for trippy, abstract, and mind-bending artistic styles."
    },
    "artificialguybr/ps1redmond-ps1-game-graphics-lora-for-sdxl": {
        "trigger_words": ["Playstation 1 Graphics", "PS1 Game"],
        "description": "Recreates retro PlayStation 1 era 3D graphics with low-poly models, pixelated textures, and nostalgic 90s gaming aesthetics."
    }
}

# Create system prompt with LoRA information
def create_system_prompt():
    lora_info = []
    for lora_name, info in AVAILABLE_LORAS.items():
        trigger_text = f" | Triggers: {', '.join(info['trigger_words'])}" if info['trigger_words'] else " | No triggers needed"
        lora_info.append(f"• {lora_name}\n  Description: {info['description']}{trigger_text}")

    return f"""
You are an expert at selecting the most appropriate LoRA adapters for SDXL image generation based on prompts.

AVAILABLE LORAS:
{chr(10).join(lora_info)}

Your task:
1. Analyze the user's prompt carefully
2. Identify key visual styles, themes, subjects, and artistic elements
3. Select only 1-2 most relevant LoRAs that would best enhance the prompt based on their DESCRIPTIONS
4. Consider style compatibility and avoid conflicting adapters
5. Use the descriptions to understand what each LoRA does and match it to the prompt's requirements

Return ONLY a JSON list of the exact LoRA names from the available list above. Select maximum 3 LoRAs.
Do NOT explain your reasoning. Do NOT add any other text.

Examples:
- "cyberpunk robot in neon city" → ["RalFinger/alien-style-lora-sdxl"]
- "hand-drawn artwork of mountains" → ["TheLastBen/Papercut_SDXL"]
- "photorealistic portrait of a woman" → ["ntc-ai/SDXL-LoRA-slider.ultra-realistic-illustration"]
- "anime character with big expressive eyes" → ["ntc-ai/SDXL-LoRA-slider.huge-anime-eyes"]
- "retro gaming character sprite" → ["artificialguybr/ps1redmond-ps1-game-graphics-lora-for-sdxl"]
"""

import json

def select_best_loras(prompt: str):
    """
    Uses Mistral to intelligently select the best LoRAs for the given prompt
    """
    try:
        # Use the standard chat completion instead of streaming
        response = client.chat.complete(
            model="open-mixtral-8x7b",
            messages=[
                {"role": "system", "content": create_system_prompt()},
                {"role": "user", "content": f"Select best LoRAs for: '{prompt}'"}
            ],
            temperature=0.1,
            max_tokens=200
        )

        # Get the response content
        response_text = response.choices[0].message.content.strip()
        print(f"🔍 Raw Mistral response: {response_text}")

        # Try to extract JSON from the response
        # Sometimes the model might add extra text, so we look for JSON array
        import re
        json_match = re.search(r'\[.*?\]', response_text, re.DOTALL)
        if json_match:
            json_text = json_match.group(0)
            selected_loras = json.loads(json_text)
        else:
            # If no JSON array found, try parsing the whole response
            selected_loras = json.loads(response_text)

        # Validate that selected LoRAs are in our available list
        valid_loras = [lora for lora in selected_loras if lora in AVAILABLE_LORAS]
        return valid_loras

    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        print(f"Response was: {response_text if 'response_text' in locals() else 'No response'}")
        return []
    except Exception as e:
        print(f"Error with Mistral API: {e}")
        return []

def enhance_prompt_with_triggers(original_prompt: str, selected_loras: list) -> str:
    """
    Enhance the original prompt by adding trigger words for selected LoRAs
    """
    enhanced_prompt = original_prompt
    added_triggers = []

    for lora_name in selected_loras:
        if lora_name in AVAILABLE_LORAS:
            trigger_words = AVAILABLE_LORAS[lora_name]["trigger_words"]
            if trigger_words:
                # Use the first trigger word for each LoRA
                primary_trigger = trigger_words[0]
                if primary_trigger.lower() not in enhanced_prompt.lower():
                    enhanced_prompt = f"{primary_trigger}, {enhanced_prompt}"
                    added_triggers.append(primary_trigger)

    if added_triggers:
        print(f"🏷  Added trigger words: {', '.join(added_triggers)}")
        print(f"📝 Enhanced prompt: '{enhanced_prompt}'")
    else:
        print("📝 No trigger words needed for selected LoRAs")

    return enhanced_prompt

import torch
from diffusers import DiffusionPipeline, LCMScheduler
import time

# Initialize the SDXL pipeline
# pipe = DiffusionPipeline.from_pretrained(
#     "stabilityai/stable-diffusion-xl-base-1.0",
#     variant="fp16",
#     torch_dtype=torch.float16
# ).to("cuda")

# pipe.scheduler = LCMScheduler()
# pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

def generate_image_with_smart_loras(pipe,prompt: str, num_inference_steps: int = 4):
    """
    Generate image with intelligently selected LoRAs and enhanced prompt
    """
    print("starting of the function generate_image_with_sma...")
    start_time = time.time()

    # Clear any existing LoRA adapters first
    print("🧹 Clearing existing LoRA adapters...")
    try:
        pipe.unload_lora_weights()
        print("✅ Existing LoRAs cleared")
    except Exception as e:
        print(f"⚠  Note: {e}")

    # Always load LCM first
    pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl", adapter_name="lcm")
    adapter_names = ["lcm"]
    adapter_weights = [1.0]

    # Get AI-selected LoRAs
    print(f"🤖 Analyzing prompt: '{prompt}'")
    selected_loras = select_best_loras(prompt)
    print(f"📋 Selected LoRAs: {selected_loras}")

    # Enhance prompt with trigger words
    enhanced_prompt = enhance_prompt_with_triggers(prompt, selected_loras)

    # Load selected LoRAs
    for i, lora_path in enumerate(selected_loras):
        adapter_name = f"lora_{i}"
        try:
            pipe.load_lora_weights(lora_path, adapter_name=adapter_name)
            adapter_names.append(adapter_name)
            adapter_weights.append(0.8)  # Slightly lower weight to avoid overpowering

            # Show LoRA info
            lora_info = AVAILABLE_LORAS.get(lora_path, {})
            triggers = lora_info.get("trigger_words", [])
            trigger_text = f" (triggers: {', '.join(triggers)})" if triggers else " (no triggers)"
            print(f"✅ Loaded: {lora_path}{trigger_text}")
        except Exception as e:
            print(f"❌ Failed to load {lora_path}: {e}")

    # Set all adapters
    if len(adapter_names) > 1:
        pipe.set_adapters(adapter_names, adapter_weights)
        print(f"🎛  Active adapters: {adapter_names} with weights: {adapter_weights}")

    # Generate image with enhanced prompt
    generator = torch.manual_seed(42)
    image = pipe(
        enhanced_prompt,  # Use enhanced prompt instead of original
        num_inference_steps=num_inference_steps,
        guidance_scale=1.0,
        generator=generator
    ).images[0]

    end_time = time.time()
    print(f"⚡ Generation completed in {end_time - start_time:.2f} seconds")
    # del pipe
    return image, selected_loras, enhanced_prompt


def create_sdxl_pipeline():
    pipe = StableDiffusionXLPipeline(
        vae=SDXL_vae,
        text_encoder=SDXL_text_encoder,
        text_encoder_2=SDXL_text_encoder_two,
        tokenizer=SDXL_tokenizer_one,
        tokenizer_2=SDXL_tokenizer_two,
        unet=SDXL_unet,
        scheduler=SDXL_scheduler
    ).to("cuda")

    pipe.enable_model_cpu_offload()  # Optional if you're memory constrained
    pipe.set_progress_bar_config(disable=True)  # Optional for cleaner logs
    return pipe
@app.post("/generate-image-automatic")
async def generate_image(
    prompt: str = Form(...),
    num_inference_steps: int = Form(4),
    seed: Optional[int] = Form(None),
    return_metadata: bool = Form(True),
):
    pipe=create_sdxl_pipeline()
    pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
    # pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
    try:
        if seed is not None:
            torch.manual_seed(seed)

        image, used_loras, final_prompt = generate_image_with_smart_loras(pipe,prompt, num_inference_steps)

        SDXL_vae.to("cpu")
        SDXL_unet.to("cpu")
        SDXL_text_encoder.to("cpu")
        SDXL_text_encoder_two.to("cpu")
        del pipe
        torch.cuda.empty_cache()
        # Convert image to base64
        buffer = io.BytesIO()
        image.save(buffer, format="PNG")
        base64_image = base64.b64encode(buffer.getvalue()).decode("utf-8")

        metadata = {
            "original_prompt": prompt,
            "final_prompt": final_prompt,
            "used_loras": used_loras,
            "lora_details": [AVAILABLE_LORAS.get(lora, {}) for lora in used_loras],
            "inference_steps": num_inference_steps,
            "seed": seed if seed is not None else "random"
        } if return_metadata else None

        return JSONResponse(content={
            "image_base64": base64_image,
            "metadata": metadata
        })

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))
import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import os

# Setup
nest_asyncio.apply()

# Kill existing ngrok tunnels
ngrok.kill()

# Kill any process on port 8000
!lsof -t -i:8000 | xargs kill -9

# Auth (do only once ideally)
!ngrok authtoken <your-auth-token>

# Start tunnel
public_url = ngrok.connect(8000)
print("🔗 Public URL:", public_url)

# Run FastAPI app
uvicorn.run(app, host="0.0.0.0", port=8000)
